In [354]:
import os
import pandas as pd
import json


pd.set_option('display.float_format', '{:.2f}'.format)


In [355]:


# Define the folder number
folder_number = '13'  # Change this to the specific folder number you are working with

# Define the paths to the JSON files
base_path = f'results_Aug1/slow_liquidity/'
channels_json_path = os.path.join(base_path, 'channels.json')
forwarding_history_json_path = os.path.join(base_path, 'forwarding_history.json')




In [356]:


# Load channels.json
with open(channels_json_path, 'r') as file:
    channels_data = json.load(file)

# Load forwarding_history.json
with open(forwarding_history_json_path, 'r') as file:
    forwarding_history_data = json.load(file)

# Extract the forwarding events
forwards = forwarding_history_data.get('forwards', [])

# Create the initial DataFrame
rows = []
for entry in forwards:
    if isinstance(entry, dict):
        rows.append({
            'addTimeNs': int(entry['addTimeNs']),
            'resolveTimeNs': int(entry.get('resolveTimeNs', 0)),
            'eventTimeNs': int(entry['addTimeNs']),
            'eventType': 'add',
            'incomingAmount': int(entry['incomingAmount']),
            'outgoingAmount': int(entry['outgoingAmount']),
            'shortChannelId_outgoing': int(entry['outgoingCircuit']['shortChannelId']),
            'shortChannelId_incoming': int(entry['incomingCircuit']['shortChannelId']),
        })
        if 'resolveTimeNs' in entry:
            rows.append({
                'addTimeNs': int(entry['addTimeNs']),
                'resolveTimeNs': int(entry['resolveTimeNs']),
                'eventTimeNs': int(entry['resolveTimeNs']),
                'eventType': 'resolve',
                'incomingAmount': int(entry['incomingAmount']),
                'outgoingAmount': int(entry['outgoingAmount']),
                'shortChannelId_outgoing': int(entry['outgoingCircuit']['shortChannelId']),
                'shortChannelId_incoming': int(entry['incomingCircuit']['shortChannelId']),
            })

pair_schedule_df = pd.DataFrame(rows)

# Sort the DataFrame by eventTimeNs
pair_schedule_df = pair_schedule_df.sort_values(by='eventTimeNs').reset_index(drop=True)

In [357]:
# Create a DataFrame for channel capacities
channels_df = pd.DataFrame(channels_data['channels'])
channels_df['chan_id'] = channels_df['chan_id'].astype(int)
channels_df['capacity'] = channels_df['capacity'].astype(int)*1000
channels_df = channels_df[['chan_id', 'capacity']]

In [ ]:
def track_funds(pair_schedule_df, channels_df, jam_lim = 0.2):
    # Initialize a dictionary to track available funds
    #funds = {channel: {'incoming': cap // 2, 'outgoing': cap // 2} for channel, cap in zip(channels_df['chan_id'], channels_df['capacity'])}
    funds = {channel: {'incoming': 0, 'outgoing': 0} for channel, cap in zip(channels_df['chan_id'], channels_df['capacity'])}

    
    # List to store times when locked funds exceed 80%
    high_locked_times = []
    status = []
    for _, row in pair_schedule_df.iterrows():
        #if (row['shortChannelId_incoming'] not in {545357767442432, 551954837209088}) and (row['shortChannelId_outgoing'] not in {545357767442432, 551954837209088}):
        if True:
            if row['eventType'] == 'add':
                funds[row['shortChannelId_incoming']]['outgoing'] += row['incomingAmount']
                funds[row['shortChannelId_outgoing']]['incoming'] += row['outgoingAmount']
            elif row['eventType'] == 'resolve':
                funds[row['shortChannelId_incoming']]['outgoing'] -= row['incomingAmount']
                funds[row['shortChannelId_outgoing']]['incoming'] -= row['outgoingAmount']
 


            # Check if the locked amounts exceed 80% of available funds
            for channel, cap in zip(channels_df['chan_id'], channels_df['capacity']):
                status.append({
                        'time': row['eventTimeNs'],
                        'channel': channel,
                        'capacity': cap,
                        'incoming_locked': funds[channel]['incoming'],
                        'outgoing_locked': funds[channel]['outgoing']
                    })
                if funds[channel]['incoming'] > jam_lim * (cap // 2):
                    high_locked_times.append({
                        'time': row['eventTimeNs'],
                        'resolve_time': row['resolveTimeNs'],
                        'channel': channel,
                        'capacity': cap,
                        'direction': 'incoming',
                        'funds_locked': funds[channel]['incoming'],
                    })
                if funds[channel]['outgoing'] > jam_lim * (cap // 2):
                    high_locked_times.append({
                        'time': row['eventTimeNs'],
                        'resolve_time': row['resolveTimeNs'],
                        'channel': channel,
                        'capacity': cap,
                        'direction': 'outgoing',
                        'funds_locked': funds[channel]['outgoing']   
                    })

    return pd.DataFrame(high_locked_times), pd.DataFrame(status)

liq_jam_ratio = 0.9
high_locked_df, st_df = track_funds(pair_schedule_df, channels_df, liq_jam_ratio)


In [ ]:
pair_schedule_df.head(10)

In [ ]:
st_df.head()

In [ ]:
high_locked_df.head(20)

In [ ]:
high_locked_df[high_locked_df['channel']==379331511648256]

# count jammed time

In [ ]:
def calculate_active_time(df):
    # Sort by start time
    df = df.sort_values(by='time')
    
    # Initialize variables
    total_active_time = 0
    current_start = None
    current_end = None
    
    for _, row in df.iterrows():
        start, end = row['time'], row['resolve_time']
        
        if current_start is None:
            current_start = start
            current_end = end
        else:
            if start <= current_end:
                # Overlapping period, extend the end time if needed
                current_end = max(current_end, end)
            else:
                # Non-overlapping period, add the previous period's duration
                total_active_time += current_end - current_start
                current_start = start
                current_end = end
    
    # Add the last period
    if current_start is not None:
        total_active_time += current_end - current_start
    
    return total_active_time

grouped = high_locked_df.groupby(['channel', 'direction'])

results = grouped.apply(calculate_active_time).reset_index()
results.columns = ['channel', 'direction', 'total_active_time']

# Step 5: Print the results
print(results)

In [ ]:
results['total_active_time_minutes'] = results['total_active_time']/60000000000

In [ ]:
results